In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow import keras
from math import sqrt
from matplotlib import pyplot as plt
import numpy
import os
import platform
import glob
import tensorflow as tf
from keras.callbacks import *
from numpy import array
from keras.layers import Dropout
import pywt

# Vieno įvesties kintamojo metodai

## Vieno žingsnio prognozės metodai

In [3]:
def atsisiusti_elspot_kainu_duomenis(aplankas, metai):
  if not os.path.exists(aplankas):
      os.mkdir(aplankas)

  for m in metai:
    !wget -P {aplankas} https://www.nordpoolgroup.com/49c730/globalassets/marketdata-excel-files-for-media/elspot-prices_{m}_hourly_eur.xls

def gauti_duomenu_rinkini(aplankas, kainos_zona):
  visi_failai = sorted(glob.glob(os.path.join(aplankas, "*.xlsx")))
  li = []
  for failo_pav in visi_failai:
    print(failo_pav)
    df = pd.read_excel(failo_pav, header = 1, skiprows=[0])
    df = df.rename(columns={'Unnamed: 0': 'Data', 'Hours': 'Valandos'})
    if f'{kainos_zona}.1' in df:
      df[kainos_zona] = df[kainos_zona].fillna(df[f'{kainos_zona}.1'])
    df = df[['Data', 'Valandos', kainos_zona]]
    li.append(df)
  duomenys = pd.concat(li, axis=0, ignore_index=True)
  return duomenys

# pašalinti duomenų sezoniškumą
def skirtumas(duomenys, intervalas=1):
  diff = list()
  for i in range(intervalas, len(duomenys)):
    verte = duomenys.iloc[i] - duomenys.iloc[i - intervalas]
    diff.append(verte)
  return Series(diff)

# sugrąžinti duomenis iš sezoniškumo pašalintų į pradinius
def invertuoti_skirtuma(istorija, yhat, intervalas=1):
  return yhat + istorija.iloc[-intervalas]

def laiko_eilute_i_priziurejima(duomenys, lag=1):
  df = DataFrame(duomenys)
  stulpeliai = [df.shift(i) for i in range(1, lag+1)]
  stulpeliai.append(df)
  df = concat(stulpeliai, axis=1)
  df.fillna(0, inplace=True)
  return df

# padalina duomenis į iš pavyzdžių, laiko žingsnių ir požymių dimensijų susidedančią matricą
def dalinti_duomenis(duomenys):
  X, y = duomenys[:, 0:-1], duomenys[:, -1]
  X = X.reshape(X.shape[0], 1, X.shape[1])
  return X, y

def duomenis_i_skale(train, test):

  scaleris = MinMaxScaler(feature_range=(-1, 1))
  scaleris = scaleris.fit(train)

  train = train.reshape(train.shape[0], train.shape[1])
  train_skaleje = scaleris.transform(train)

  test = test.reshape(test.shape[0], test.shape[1])
  test_skaleje = scaleris.transform(test)

  return scaleris, train_skaleje, test_skaleje

def duomenis_is_skales(scaleris, X, verte):
  nauja_eilute = [x for x in X] + [verte]
  array = numpy.array(nauja_eilute)
  array = array.reshape(1, len(array))
  invertuota = scaleris.inverse_transform(array)
  return invertuota[0, -1]

def mokyti_ITAMNT(train, rinkinio_dydis, epizodu_sk, neuronu_sk):
  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU nerastas')
  print('GPU naudojamas iš: {}'.format(device_name))
  X, y = dalinti_duomenis(train)
  # X_val, y_val = dalinti_duomenis(val) # kol kas be val bandau
  print(f'pavyzdžiai: {rinkinio_dydis}, laiko_žingsniai: {X.shape[1]}, požymiai: {X.shape[2]}')
  modelis = Sequential()
  modelis.add(LSTM(neuronu_sk, batch_input_shape = (rinkinio_dydis, X.shape[1], X.shape[2]), stateful=True))
  modelis.add(Dense(1))
  modelis.compile(loss='mean_squared_error', optimizer='adam')
  for i in range(epizodu_sk):
    print("epocha=" + str(i))
    modelis.fit(X, y, epochs=1, batch_size=rinkinio_dydis, verbose=1, shuffle=False)
    modelis.reset_states()
  return modelis

def prognoze(modelis, rinkinio_sk, eile):
  X = eile[0: -1]
  X = X.reshape(1, 1, len(X))
  yhat = modelis.predict(X, batch_size=rinkinio_sk)
  return yhat[0,0]

def itamnt_prognoze(modelis, rinkinio_dydis, X):
  X = X.reshape(1,1, len(X))
  yhat = modelis.predict(X, batch_size=rinkinio_dydis)
  return yhat[0,0]

def prognozuoti(modelis, test_skaleje, raw_duomenys, scaleris):
  prognozes = list()
  for i in range(len(test_skaleje)):
    print(f'{str(i+1)} iš {str(len(test_skaleje))}')
    X, y = test_skaleje[i, 0: -1], test_skaleje[i, -1]
    yhat = itamnt_prognoze(modelis, 1, X)
    yhat = duomenis_is_skales(scaleris, X, yhat)
    yhat = invertuoti_skirtuma(raw_duomenys, yhat, len(test_skaleje)+1-i)
    prognozes.append(yhat)
  return prognozes

def paruosti_duomenis_duotame_tarpe(duomenys, kainos_zona, data_nuo, test_sk_procentais):
  duom = duomenys[duomenys['Data'].eq(data_nuo).cummax()]
  print("Imami duomenys nuo...")
  print(duom.head())
  duom = duom[kainos_zona].dropna()

  duomenys_be_sezonaliskumo = skirtumas(duom)
  duom_priziurejimui = laiko_eilute_i_priziurejima(duomenys_be_sezonaliskumo)
  priz_vertes = duom_priziurejimui.values
  test_dydis = round(test_sk_procentais * priz_vertes.shape[0])
  print(f"Duomenų skaičius validavimui: {str(test_dydis)} Tai yra {str(test_dydis/24)} dienos")
  train_dydis = priz_vertes.shape[0]-test_dydis
  print(f"Duomenų skaičius treniravimui: {str(train_dydis)} Tai yra {str(train_dydis/24)} dienos")
  train, test = priz_vertes[0:train_dydis,:], priz_vertes[train_dydis:,:]
  scaleris, train_skaleje, test_skaleje = duomenis_i_skale(train, test)
  
  return duom, train_skaleje, test_skaleje, scaleris

def diagrama(dienos, pavadinimas, kainos_zona, realios_kainos, prognozes):
  
  rodoma_dalis = 24*dienos
  print(f'Vaizduojama nuo: {realios_kainos[-rodoma_dalis:]}')
  plt.figure(figsize=(15,7))
  plt.title(pavadinimas)
  plt.xlabel('Valanda')
  plt.ylabel(f'Elektros kaina [EUR] {kainos_zona} zonoje')
  plt.plot(range(rodoma_dalis),realios_kainos[-rodoma_dalis:],color='b')
  plt.plot(range(rodoma_dalis),prognozes[-rodoma_dalis:], color='r')
  plt.legend(['reali kaina', 'prognoze'])
  plt.show()

def prognozes_tikslumas(test_skaleje, realios_kainos, prognozes):
  test_zymos = test_skaleje.shape[0]
  rmse = sqrt(mean_squared_error(realios_kainos[-test_zymos:], prognozes))
  mae = mean_absolute_error(realios_kainos[-test_zymos:], prognozes)
  print(f'VKNŠ = {rmse}')
  print(f'VAP = {mae}')



## Kelių žingsnių prognozės metodai

In [ ]:
def gauti_duomenu_rinkini(aplankas, kainos_zona):
  visi_failai = sorted(glob.glob(os.path.join(aplankas, "*.xlsx")))
  li = []
  for failo_pav in visi_failai:
    print(failo_pav)
    df = pd.read_excel(failo_pav, header = 1, skiprows=[0])
    df = df.rename(columns={'Unnamed: 0': 'Data', 'Hours': 'Valandos'})
    if f'{kainos_zona}.1' in df:
      df[kainos_zona] = df[kainos_zona].fillna(df[f'{kainos_zona}.1'])
    df = df[['Data', 'Valandos', kainos_zona]]
    li.append(df)
  duomenys = pd.concat(li, axis=0, ignore_index=True)
  return duomenys

# pašalinti duomenų sezoniškumą
def skirtumas(duomenys, intervalas=1):
  diff = list()
  for i in range(intervalas, len(duomenys)):
    verte = duomenys.iloc[i] - duomenys.iloc[i - intervalas]
    diff.append(verte)
  return Series(diff)

def invertuoti_ms_skirtuma(paskutine_reiksme, prognoze):
  invertuoti = list()
  print(f'Prognozuojama nuo šios paskutinės reikšmės: {paskutine_reiksme}')
  invertuoti.append(prognoze[0] + paskutine_reiksme)
  for i in range(1, len(prognoze)):
    invertuoti.append(prognoze[i] + invertuoti[i-1])
  return invertuoti

def invertuoti_transformacijas(duomenys, prognozes, scaleris, test_dydis, koeficientai=""):
  invertuoti = list()
  for i in range(len(prognozes)):
    prognoze = array(prognozes[i])
    prognoze = prognoze.reshape(1, len(prognoze))

    invertuota_skale = scaleris.inverse_transform(prognoze)
    invertuota_skale = invertuota_skale[0, :]

    index = len(duomenys) - test_dydis + i #- 1
    paskutine_reiksme = duomenys.values[index]
    invertuotos = invertuoti_ms_skirtuma(paskutine_reiksme, invertuota_skale)
    invertuoti.append(invertuotos)
  return invertuoti

def laiko_eilute_i_priziurejima_patobulinta(data, n_ivestis=1, n_isvestis=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	stulp, pav = list(), list()
	# įvesties seka (t-n, ... t-1)
	for i in range(n_ivestis, 0, -1):
		stulp.append(df.shift(i))
		pav += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# išvesties seka (t, t+1, ... t+n)
	for i in range(0, n_isvestis):
		stulp.append(df.shift(-i))
		if i == 0:
			pav += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			pav += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	agg = concat(stulp, axis=1)
	agg.columns = pav
	if dropnan:
		agg.dropna(inplace=True)
	return agg


def paruosti_duomenis_duotame_tarpe_ms(duomenys, kainos_zona, data_nuo, data_iki, n_ivestis, n_isvestis, test_dydis):#, test_sk_procentais):

  duom_pjuvis = duomenys[(duomenys['Data'].eq(data_nuo).cummax()) & (duomenys['Data'] <= data_iki)]
  print(f'Imami duomenys nuo:\n{duom_pjuvis.head()}\n iki:\n{duom_pjuvis.tail()}')
  # print(f'Prognozojuoma {test_dydis} valandas į priekį bus nuo kiekvieno iš šių taškų:\n {duom_pjuvis[-test_dydis:]}')
  duom = duom_pjuvis[kainos_zona].dropna()

  print("Naudojama skirtumo f-ja duomenims paruošti")
  duomenys_be_sezonaliskumo = skirtumas(duom)
  duomenys_be_sezonaliskumo_vertes = duomenys_be_sezonaliskumo.values
  dbsv = duomenys_be_sezonaliskumo_vertes.reshape(len(duomenys_be_sezonaliskumo_vertes), 1)
  koeficientai = []

  scaleris = MinMaxScaler(feature_range=(-1, 1))
  vertes_skaleje = scaleris.fit_transform(dbsv)
  vertes_skaleje = vertes_skaleje.reshape(len(vertes_skaleje), 1)

  duom_priziurejimui = laiko_eilute_i_priziurejima_patobulinta(vertes_skaleje, n_ivestis, n_isvestis)
  priz_vertes = duom_priziurejimui.values
  print(f"t+{n_isvestis} prognozei skirti laiko taškai: {str(test_dydis)}")
  print(f"Pirmasis prognozės taškas:\n {duom_pjuvis.iloc[-test_dydis:]}")
  train_dydis = priz_vertes.shape[0]-test_dydis
  print(f"Duomenų skaičius treniravimui: {str(train_dydis)} Tai yra {str(train_dydis/24)} dienos")
  train, test = priz_vertes[0:train_dydis,:], priz_vertes[train_dydis:,:]

  realios_test_reiksmes = duom_pjuvis.iloc[-test_dydis:]
  prognozuojamu_tasku_pradzia = (realios_test_reiksmes.index)[0]
  prognozuojamu_reiksmiu_realios_reiksmes = duomenys.iloc[prognozuojamu_tasku_pradzia:prognozuojamu_tasku_pradzia+n_isvestis*2]

  return duom, train, test, scaleris, prognozuojamu_reiksmiu_realios_reiksmes, koeficientai


def mokyti_ms_itamnt(train, n_ivestis, n_isvestis, rinkinio_dydis, epizodu_sk, neuronu_sk, checkpoint_kelias=""):

  saugoti_kas = 30
  saugojimas = 0
  X, y = train[:, 0: n_ivestis], train[:, n_ivestis:]
  X = X.reshape(X.shape[0], 1, X.shape[1])

  modelis = Sequential()
  modelis.add(LSTM(neuronu_sk, batch_input_shape=(rinkinio_dydis, X.shape[1], X.shape[2]), stateful=True, return_sequences=True))
  modelis.add(Dropout(0.2))
  modelis.add(LSTM(neuronu_sk, stateful=True))
  modelis.add(Dropout(0.2))
  modelis.add(Dense(y.shape[1]))
  modelis.compile(loss='mean_squared_error', optimizer='adam')
  # if(checkpoint_kelias != ""):
  #   print(f'Imamas modelis iš {checkpoint_kelias}')
  #   modelis = keras.models.load_model(checkpoint_kelias)

  for i in range(epizodu_sk):
    print(f'epizodas: {i+1}/{epizodu_sk}')

    if ((i+1) % saugoti_kas) == 0:
      modelis.save(f'{checkpoint_kelias}/epocha{i+1}')

    modelis.fit(X, y, epochs=1, batch_size=rinkinio_dydis, shuffle=False)
    modelis.reset_states()
  return modelis

def ms_itamnt_prognoze(modelis, X, rinkinio_dydis):
  X = X.reshape(1,1,len(X))
  prognoze = modelis.predict(X, batch_size=rinkinio_dydis)
  return [p for p in prognoze[0, :]]

def ms_prognoze(modelis, rinkinio_dydis, test, n_ivestis, n_isvestis):
  prognozes = list()
  for i in range(len(test)):
    X, y = test[i, 0:n_ivestis], test[i, n_ivestis:]
    prognoze = ms_itamnt_prognoze(modelis, X, rinkinio_dydis)
    prognozes.append(prognoze)
  return prognozes

def ms_diagrama(realios_kainos, prognozes, kainos_zona, pavadinimas="", top_indexai=None, top3=False):

  realios_vertes = realios_kainos[kainos_zona].values
  print(realios_vertes)
  x = np.arange(len(realios_vertes))
  prognozes_arr = np.array(prognozes)

  plt.figure(figsize=(15,7))
  plt.plot(x, realios_vertes, label='Realios kainos', marker='o', linestyle='-')

  for i in range(len(prognozes)):#(prognozes_arr.shape[1]):
    
    if top3:
      if i in top_indexai:
            x_pred = np.arange(i+1, i + 1 +  prognozes_arr.shape[1])
            plt.plot(x_pred, prognozes_arr[i], label=f'Prognozė {i+1}', marker='o', linestyle='--')
      else:
        continue
    else:
      x_pred = np.arange(i+1, i + 1 +  prognozes_arr.shape[1])
      plt.plot(x_pred, prognozes_arr[i], label=f'Prognozė {i+1}', marker='o', linestyle='--')

  plt.legend()
  plt.xlabel('Valandos')
  plt.ylabel('Kaina')
  plt.title(pavadinimas)
  plt.show()

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sqrt(np.mean((y_true - y_pred)**2))

def prognozes_tikslumo_metrikos(realios, prognozes, kainos_zona):
  
  realios_vertes = realios[kainos_zona].values
  prognozes_arr = np.array(prognozes)
  n_isvestis = prognozes_arr.shape[1]
  errors = []

  for i in range(len(prognozes_arr)):
      # print(prognozes_arr[i])
      # print(realios_vertes[i+1:i+n_isvestis+1])
      rmse = root_mean_squared_error(realios_vertes[i+1:i+n_isvestis+1], prognozes_arr[i])
      mape = mean_absolute_percentage_error(realios_vertes[i+1:i+n_isvestis+1], prognozes_arr[i])

      errors.append((rmse, mape))
      
  for i, (rmse, mape) in enumerate(errors):
      print(f"Prognozė {i + 1}: VKNŠ = {rmse:.2f}, VAPP = {mape:.2f}%")

  return errors


# Naivus prognozuotojas

In [2]:
def naivus_prognozuotojas(indexas_nuo, indexas_iki, valandos, diagramos_title):

  naive_data = old_duomenys[indexas_nuo-valandos:indexas_iki]
  naive_plot = old_duomenys[indexas_nuo:indexas_iki]

  naive_data = naive_data[kainos_zona].dropna()
  naive_plot = naive_plot[kainos_zona].dropna()

  naive_data = naive_data.shift(valandos).dropna()

  plt.figure(figsize=(15, 7))
  plt.plot(range(len(naive_plot)), naive_plot, label='Reali kaina', color='b')
  plt.plot(range(len(naive_data)), naive_data, label='Naivi prognozė', color='r')
  plt.xlabel('Valandos')
  plt.ylabel('Elektros kaina')
  plt.legend()
  plt.title(diagramos_title)
  plt.show()


# Kelių įvesties kintamųjų metodai

## SDV Failų apdirbimas



In [4]:
def paruosti_sdv_operating_data(sdv_failas):

  sdv_data = read_csv(sdv_failas, sep=';', header = None, skiprows=14, names=["Data type", "Code", "Year", "Week", "Day", "Date", "Currency", "Hour1","Hour2","Hour3","Hour3B","Hour4","Hour5","Hour6","Hour7","Hour8","Hour9","Hour10","Hour11","Hour12","Hour13","Hour14","Hour15","Hour16","Hour17","Hour18","Hour19","Hour20","Hour21","Hour22","Hour23","Hour24", "Sum"])
  sdv_data = sdv_data.drop(columns=['Data type', "Year", "Week", "Day", "Currency", "Hour3B", "Sum"], axis = 1)
  sdv_data = sdv_data.loc[sdv_data['Code'].isin(['F', 'P'])]
  sdv_melted = sdv_data.melt(id_vars=['Code', 'Date'], var_name = 'Hour', value_name='Value')
  sdv_melted['Hour'] = sdv_melted['Hour'].str.replace('Hour', '').astype(int) - 1
  sdv_pivoted = sdv_melted.pivot_table(values='Value', index=['Date', 'Hour'], columns='Code').reset_index()
  sdv_pivoted['Date'] = pd.to_datetime(sdv_pivoted['Date'], format='%d.%m.%Y') + pd.to_timedelta(sdv_pivoted['Hour'], unit='h')
  sdv_pivoted.set_index('Date', inplace=True)
  sdv_pivoted = sdv_pivoted.drop(columns=['Hour'], axis=1)
  sdv_pivoted = sdv_pivoted.rename(columns={'F': 'Elektros sunaudojimas', 'P': 'Elektros pagaminimas'})
  sdv_pivoted = sdv_pivoted.sort_index()

  return sdv_pivoted

def paruosti_sdv_maksimalius_perdavimus(sdv_capacity):

  sdv_cap = read_csv(sdv_capacity, sep = ';', header = None, skiprows=34, encoding="ISO-8859-1", names=['Data type', 'Flow', "Year", "Week", "Day", "Date", "Zones", "Hour1","Hour2","Hour3","Hour3B","Hour4","Hour5","Hour6","Hour7","Hour8","Hour9","Hour10","Hour11","Hour12","Hour13","Hour14","Hour15","Hour16","Hour17","Hour18","Hour19","Hour20","Hour21","Hour22","Hour23","Hour24", "Sum"])
  sdv_cap = sdv_cap.drop(columns=['Data type', 'Flow', "Year", "Week", "Day", "Hour3B", "Sum" ])
  sdv_cap = sdv_cap.dropna()
  sdv_cap = sdv_cap[sdv_cap['Zones'].str.endswith('_LT')]
  sdv_cap_melted = sdv_cap.melt(id_vars=['Date', 'Zones'], var_name = 'Hour', value_name='Value')
  sdv_cap_melted['Hour'] = sdv_cap_melted['Hour'].str.replace('Hour', '').astype(int) - 1
  sdv_cap_pivoted = sdv_cap_melted.pivot_table(values='Value', index=['Date', 'Hour'], columns='Zones').reset_index()
  sdv_cap_pivoted['Date'] = pd.to_datetime(sdv_cap_pivoted['Date'], format='%d.%m.%Y') + pd.to_timedelta(sdv_cap_pivoted['Hour'], unit='h')
  sdv_cap_pivoted.set_index(['Date'],inplace=True)
  sdv_cap_pivoted = sdv_cap_pivoted.drop(columns=['Hour'], axis=1)
  sdv_cap_pivoted = sdv_cap_pivoted.add_suffix('_maksimalus_perdavimo_pajegumas')
  sdv_cap_pivoted = sdv_cap_pivoted.sort_index()

  return sdv_cap_pivoted

def paruosti_sdv_dabartinius_perdavimus(sdv_flow):

  flow = read_csv(sdv_flow, sep=';', header = None, skiprows=35, encoding="ISO-8859-1", decimal=',', names=['Data type', 'Flow', "Year", "Week", "Day", "Date", "Zones", "Hour1","Hour2","Hour3","Hour3B","Hour4","Hour5","Hour6","Hour7","Hour8","Hour9","Hour10","Hour11","Hour12","Hour13","Hour14","Hour15","Hour16","Hour17","Hour18","Hour19","Hour20","Hour21","Hour22","Hour23","Hour24", "Sum"])
  flow = flow.fillna(0.0)
  flow = flow.head(-1)
  flow = flow.drop(columns=['Data type', 'Flow', "Year", "Week", "Day", "Hour3B", "Sum" ])
  flow = flow[flow['Zones'].str.endswith('_LT', na=False)]
  flow_melted = flow.melt(id_vars=['Date', 'Zones'], var_name = 'Hour', value_name='Value')
  flow_melted['Hour'] = flow_melted['Hour'].str.replace('Hour', '').astype(int) - 1
  flow_pivoted = flow_melted.pivot_table(values='Value', index=['Date', 'Hour'], columns='Zones').reset_index()
  flow_pivoted['Date'] = pd.to_datetime(flow_pivoted['Date'], format='%d.%m.%Y') + pd.to_timedelta(flow_pivoted['Hour'], unit='h')
  flow_pivoted.set_index(['Date'],inplace=True)
  flow_pivoted = flow_pivoted.drop(columns=['Hour'], axis=1)
  flow_pivoted = flow_pivoted.add_suffix('_dabartinis_perdavimo_pajegumas')
  flow_pivoted = flow_pivoted.sort_index()

  return flow_pivoted


def paruosti_sdv_is_aplanko(aplankas, duomenu_tipas):
  visi_failai = sorted(glob.glob(os.path.join(aplankas, "*.sdv")))
  li = []
  if duomenu_tipas == 'operaciniai':
    metodas = paruosti_sdv_operating_data
  if duomenu_tipas == 'maksimalus_perdavimai':
    metodas = paruosti_sdv_maksimalius_perdavimus
  if duomenu_tipas == 'dabartiniai_perdavimai':
    metodas = paruosti_sdv_dabartinius_perdavimus

  for failo_pav in visi_failai:
    print(failo_pav)
    sdv = metodas(failo_pav)
    li.append(sdv)

  return li



### Sujungimas į vieną bendrą failą (pavyzdys su 2023 metais):

In [ ]:
operaciniai_li = paruosti_sdv_is_aplanko('/content/drive/MyDrive/Bakalauras/Nordpool_sdv_duomenys/Lithuania/2023', 'operaciniai')
maksimalus_perdavimai_li = paruosti_sdv_is_aplanko('/content/drive/MyDrive/Bakalauras/Nordpool_sdv_duomenys/Capacity/2023', 'maksimalus_perdavimai')
dabartiniai_perdavimai_li = paruosti_sdv_is_aplanko('/content/drive/MyDrive/Bakalauras/Nordpool_sdv_duomenys/Flow/2023', 'dabartiniai_perdavimai')

operaciniai_df = pd.concat(operaciniai_li, axis=0)
maksimalus_df = pd.concat(maksimalus_perdavimai_li, axis=0)
maksimalus_df = maksimalus_df.fillna(0.0)

dabartiniai_df = pd.concat(dabartiniai_perdavimai_li, axis=0)
dabartiniai_df = dabartiniai_df.fillna(0.0)

bendras_df = pd.concat([operaciniai_df, maksimalus_df, dabartiniai_df], axis=1)

bendras_df = bendras_df['2020-01-01 00:00:00':]

nuo = bendras_df.head(1).index[0].strftime('%Y-%m-%d %H:%M:%S')
iki = bendras_df.tail(1).index[0].strftime('%Y-%m-%d %H:%M:%S')

kainos_zona = 'LT'
kainos = gauti_duomenu_rinkini('/content/drive/MyDrive/Bakalauras/Nordpool_duomenys', 'LT')

kainos['Valandos'] = kainos['Valandos'].apply(lambda x: int(x.split('-')[0]))

kainos['Data'] = pd.to_datetime(kainos['Data'] + ' ' + kainos['Valandos'].astype(str) + ':00', format="%d-%m-%Y %H:%M")

kainos = kainos.drop(columns=['Valandos'])

kainos = kainos.set_index('Data')

kainos_df = kainos.sort_index()

kainos_df = kainos_df.rename(columns={'LT':'Kaina'})

kainos_df = kainos_df[nuo:iki]

pilnai_bendras_su_kainomis_df = pd.concat([kainos_df, bendras_df], axis=1)

## Kelių įvesties kintamųjų ITAMNT metodai

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
def sukurti_duomenu_rinkini(duomenys, n_ivestis=72, n_isvestis=24):

  X, Y = [], []
  for i in range(len(duomenys) - n_ivestis - n_isvestis + 1):
    X.append(duomenys[i:(i + n_ivestis), 1:])
    Y.append(duomenys[i + n_ivestis:i + n_ivestis + n_isvestis, 0])

  return np.array(X), np.array(Y)

def duomenys_multivariate_prognozei(duomenys, n_ivestis=72, n_isvestis=24, train_procentai=0.8):

  data  = duomenys.values
  scaleris_x = MinMaxScaler(feature_range=(0, 1))
  scaleris_y = MinMaxScaler(feature_range=(0, 1))

  scaled_data_x = scaleris_x.fit_transform(data[:, 1:])
  scaled_data_y = scaleris_y.fit_transform(data[:, 0].reshape(-1, 1))
  scaled_data = np.hstack((scaled_data_y, scaled_data_x))

  X, Y = sukurti_duomenu_rinkini(scaled_data, n_ivestis, n_isvestis)

  train_dydis = int(len(X) * train_procentai)
  X_train, Y_train = X[:train_dydis], Y[:train_dydis]
  X_test, Y_test = X[train_dydis:], Y[train_dydis:]

  # Perdaryti duomenys į LSTM sluoksniams reikiamus (samples, timesteps, features)
  X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
  X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

  return X_train, Y_train, X_test, Y_test, scaleris_y

def mokyti_multivariate_itamnt(X_train, Y_train, epizodu_sk, neuronu_sk, n_isvestis):

  model = Sequential()
  model.add(LSTM(neuronu_sk, input_shape=(X_train.shape[1], X_train.shape[2])))
  model.add(Dense(n_isvestis))
  optimizer = Adam(learning_rate=0.001)
  model.compile(loss='mean_squared_error', optimizer=optimizer)

  model.fit(X_train, Y_train, epochs=epizodu_sk, batch_size=1, verbose=1, shuffle=False)

  return model

def multivariate_prognoze(modelis, X_duomenys, Y_duomenys, scaleris_y):

  prog = modelis.predict(X_duomenys)
  prog = scaleris_y.inverse_transform(prog)

  orig_duom = scaleris_y.inverse_transform(Y_duomenys)

  return orig_duom, prog

def prognoziu_tikslumas(orig, prog):

  rmse = np.sqrt(mean_squared_error(orig, prog))
  print('VKNŠ:', rmse)

def palyginamasis_df(orig, prog):

  palyginimo_df = pd.DataFrame({'Reali': orig.flatten(), 'Prognozė': prog.flatten()})
  print(f'Pirmosios 24h:\n{palyginimo_df.head(24)}')

  return palyginimo_df

def multivariate_diagrama(orig, prog, pavadinimas):

  plt.figure(figsize=(15, 8))
  plt.plot(orig[:, 0], label='Reali kaina')
  plt.plot(prog[:, 0], label='Prognozė')
  plt.xlabel('Valandos')
  plt.ylabel('Kaina EUR/MWh')
  plt.title(pavadinimas)
  plt.legend()
  plt.show()
